In [508]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder, MultiLabelBinarizer, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanSquaredError, Accuracy
from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Concatenate, Input, Dropout,  LSTM, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.regularizers import l1_l2
from keras.models import Sequential
from gensim.models import Word2Vec


In [509]:
df = pd.read_csv('/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/data/clean_cards.csv')
df.sample(5)

,cmc,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,rarity,reserved,released_at,booster,usd
22553,2,['G'],False,['Sorcery'],['none'],"Search your library for a Forest card, put it ...",['no keywords'],0,0,uncommon,False,10838,False,5.92
3390,4,['G'],False,['Sorcery'],['none'],Destroy target noncreature permanent. Then tha...,['no keywords'],0,0,uncommon,False,3232,True,0.25
13722,3,['U'],False,['Artifact'],['Vehicle'],Flying\nWhenever Mindlink Mech becomes crewed ...,"['Flying', 'Crew']",4,3,rare,False,10306,True,0.12
20656,7,['U'],False,['Enchantment'],['Aura'],Enchant creature\nYou control enchanted creatu...,['Enchant'],0,0,rare,False,6729,True,0.16
11307,4,"['R', 'U']",True,['Creature'],"['Human', 'Artificer']","Whenever you cast a historic spell, draw a car...",['no keywords'],3,3,rare,False,9746,True,2.43


In [510]:
no_prices_df = df[df['usd'].isna()]

In [511]:
df.drop(df[df['usd'].isna()].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [512]:
df['colors'] = df['colors'].apply(literal_eval)

In [513]:
mlb = MultiLabelBinarizer()
df= df.join(pd.DataFrame(mlb.fit_transform(df['colors']),
                                      columns=mlb.classes_,
                                      index=df.index))
legend_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['legendary']), columns=['legendary'])
             .add_prefix('_'))
booster_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['booster']), columns=['booster'])
             .add_prefix('_'))
reserved_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['reserved']), columns=['resrved'])
             .add_prefix('_'))

df = df.drop(['legendary', 'booster', 'reserved', 'colors'], axis=1)
df.head(3)

,cmc,types,sub_types,oracle_text,keywords,power,toughness,rarity,released_at,usd,B,C,G,N,R,U,W,_legendary,_booster,_resrved
0,2,['Artifact'],['Equipment'],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,['Equip'],0,0,common,10096,0.02,0,0,0,0,0,0,1,0,1,0
1,4,['Creature'],"['Bird', 'Rogue']",Flying\nWhen Aarakocra Sneak enters the battle...,['Flying'],1,4,common,10418,0.06,0,0,0,0,0,1,0,0,1,0
2,5,['Creature'],"['Astartes', 'Warrior']",Trample\nMark of Chaos Ascendant — During your...,"['Mark of Chaos Ascendant', 'Trample']",5,5,mythic,10537,2.81,1,0,0,0,1,1,0,1,0,0


In [514]:
df['oracle_text'] = df['oracle_text'].replace('\n', ' ', regex=True)
df['oracle_text'][0]

'Equipped creature gets +2/+2. Equip {3} ({3}: Attach to target creature you control. Equip only as a sorcery.)'

## Text Processing

####  1. Types processing

In [515]:
df['types'].value_counts()

types
['Creature']                        12870
['Instant']                          2998
['Sorcery']                          2766
['Enchantment']                      2747
['Artifact']                         1759
['Artifact', 'Creature']              907
['Land']                              788
['Planeswalker']                      267
['Enchantment', 'Creature']           170
['Snow', 'Creature']                   56
['World', 'Enchantment']               26
['Snow', 'Land']                       21
['Tribal', 'Instant']                  20
['Artifact', 'Land']                   18
['Tribal', 'Sorcery']                  16
['Tribal', 'Enchantment']              13
['Snow', 'Enchantment']                 8
['Tribal', 'Artifact']                  6
['Basic', 'Land']                       6
['Enchantment', 'Artifact']             5
['Snow', 'Sorcery']                     5
['Basic', 'Snow', 'Land']               5
['Snow', 'Artifact']                    4
['Snow', 'Artifact', 'Creatu

In [516]:
types_data = df['types']
types_list = types_data.apply(eval)

In [517]:
mlb = MultiLabelBinarizer()
encoded_data = mlb.fit_transform(types_list)
encoded_df = pd.DataFrame(encoded_data, columns=mlb.classes_)
encoded_df.head(3)

,Artifact,Basic,Creature,Enchantment,Instant,Land,Planeswalker,Snow,Sorcery,Tribal,World
0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0


In [518]:
df = pd.concat([df, encoded_df], axis=1).drop('types', axis=1)

In [519]:
df.head(3)

,cmc,sub_types,oracle_text,keywords,power,toughness,rarity,released_at,usd,B,...,Basic,Creature,Enchantment,Instant,Land,Planeswalker,Snow,Sorcery,Tribal,World
0,2,['Equipment'],Equipped creature gets +2/+2. Equip {3} ({3}: ...,['Equip'],0,0,common,10096,0.02,0,...,0,0,0,0,0,0,0,0,0,0
1,4,"['Bird', 'Rogue']",Flying When Aarakocra Sneak enters the battlef...,['Flying'],1,4,common,10418,0.06,0,...,0,1,0,0,0,0,0,0,0,0
2,5,"['Astartes', 'Warrior']",Trample Mark of Chaos Ascendant — During your ...,"['Mark of Chaos Ascendant', 'Trample']",5,5,mythic,10537,2.81,1,...,0,1,0,0,0,0,0,0,0,0


#### 2. Keywords Processing

In [520]:
df['keywords'].value_counts()

keywords
['no keywords']                    13778
['Flying']                          1349
['Enchant']                          920
['Equip']                            347
['Trample']                          328
                                   ...  
['Flying', 'Vigilance', 'Crew']        1
['Treasure', 'Raid']                   1
['Haste', 'Chroma']                    1
['Indestructible', 'Haste']            1
['Body-print']                         1
Name: count, Length: 1792, dtype: int64

#### Too many values to be binarized

In [521]:
df['keywords']=df['keywords'].apply(literal_eval)

In [522]:
keywords_model = Word2Vec(sentences=df['keywords'], vector_size=100, window=5, min_count=1, workers=4)

In [523]:
def get_average_embedding(row):
    embeddings = [keywords_model.wv[word] for word in row if word in keywords_model.wv]
    return np.mean(embeddings, axis=0) if embeddings else np.zeros(keywords_model.vector_size)

In [524]:
df['keywords_embedding'] = df['keywords'].apply(get_average_embedding)

In [525]:
df.head(3)

,cmc,sub_types,oracle_text,keywords,power,toughness,rarity,released_at,usd,B,...,Creature,Enchantment,Instant,Land,Planeswalker,Snow,Sorcery,Tribal,World,keywords_embedding
0,2,['Equipment'],Equipped creature gets +2/+2. Equip {3} ({3}: ...,[Equip],0,0,common,10096,0.02,0,...,0,0,0,0,0,0,0,0,0,"[-0.061374895, 0.13885097, 0.01970111, 0.05739..."
1,4,"['Bird', 'Rogue']",Flying When Aarakocra Sneak enters the battlef...,[Flying],1,4,common,10418,0.06,0,...,1,0,0,0,0,0,0,0,0,"[-0.14265281, 0.27659872, 0.045060355, 0.11373..."
2,5,"['Astartes', 'Warrior']",Trample Mark of Chaos Ascendant — During your ...,"[Mark of Chaos Ascendant, Trample]",5,5,mythic,10537,2.81,1,...,1,0,0,0,0,0,0,0,0,"[-0.06767147, 0.1424448, 0.02237789, 0.0555035..."


#### 3. Sub-types prcoessing

In [526]:
df['sub_types'].value_counts()

sub_types
['none']                                9108
['Aura']                                1048
['Human', 'Wizard']                      481
['Human', 'Soldier']                     456
['Equipment']                            417
                                        ... 
['Equipment', 'Cat']                       1
['Food', 'Clue']                           1
['Elemental', 'Insect']                    1
['Equipment', 'Lizard']                    1
['Alien', 'Shapeshifter', 'Soldier']       1
Name: count, Length: 2026, dtype: int64

In [527]:
df['sub_types']=df['sub_types'].apply(literal_eval)

In [528]:
sub_types_model = Word2Vec(sentences=df['sub_types'], vector_size=100, window=5, min_count=1, workers=4)

In [529]:
def get_average_embedding(row):
    embeddings = [sub_types_model.wv[word] for word in row if word in sub_types_model.wv]
    return np.mean(embeddings, axis=0) if embeddings else np.zeros(sub_types_model.vector_size)
df['sub_types_embedding'] = df['sub_types'].apply(get_average_embedding)

#### 4. Prcoess oracle text

In [ ]:
texts= df['orac_text'].astype(str).tolist()

#### The Pipeline and neural network

In [530]:
df.head(3)

,cmc,sub_types,oracle_text,keywords,power,toughness,rarity,released_at,usd,B,...,Enchantment,Instant,Land,Planeswalker,Snow,Sorcery,Tribal,World,keywords_embedding,sub_types_embedding
0,2,[Equipment],Equipped creature gets +2/+2. Equip {3} ({3}: ...,[Equip],0,0,common,10096,0.02,0,...,0,0,0,0,0,0,0,0,"[-0.061374895, 0.13885097, 0.01970111, 0.05739...","[-0.020557145, 0.064449914, -0.004561723, -0.0..."
1,4,"[Bird, Rogue]",Flying When Aarakocra Sneak enters the battlef...,[Flying],1,4,common,10418,0.06,0,...,0,0,0,0,0,0,0,0,"[-0.14265281, 0.27659872, 0.045060355, 0.11373...","[-0.026384331, 0.13771231, 0.011705989, -0.149..."
2,5,"[Astartes, Warrior]",Trample Mark of Chaos Ascendant — During your ...,"[Mark of Chaos Ascendant, Trample]",5,5,mythic,10537,2.81,1,...,0,0,0,0,0,0,0,0,"[-0.06767147, 0.1424448, 0.02237789, 0.0555035...","[-0.030232653, 0.11610664, 0.005136263, -0.129..."


In [94]:
X = df.drop(columns='usd')
y = df['usd']

In [95]:
numerical_features = ['power', 'toughness', 'released_at', 'cmc']
categorical_features = ['rarity']
remaining_features = X.drop(columns=['rarity', 'power', 'toughness', 'released_at', 'cmc'])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'
)

pipeline = make_pipeline(preprocessor)

In [96]:
X_preprocessed= pipeline.fit_transform(X)

In [97]:
column_names = pipeline.get_feature_names_out()
column_names

array(['num__power', 'num__toughness', 'num__released_at', 'num__cmc',
       'cat__rarity_common', 'cat__rarity_mythic', 'cat__rarity_rare',
       'cat__rarity_special', 'cat__rarity_uncommon', 'remainder__B',
       'remainder__C', 'remainder__G', 'remainder__N', 'remainder__R',
       'remainder__U', 'remainder__W', 'remainder___legendary',
       'remainder___booster', 'remainder___resrved',
       'remainder__text_embedding', 'remainder__types_embedding',
       'remainder__subtypes_embedding', 'remainder__keywords_embedding'],
      dtype=object)

In [98]:
processed_df = pd.DataFrame(X_preprocessed, columns=column_names)

In [99]:
processed_df.head(3)

,num__power,num__toughness,num__released_at,num__cmc,cat__rarity_common,cat__rarity_mythic,cat__rarity_rare,cat__rarity_special,cat__rarity_uncommon,remainder__B,...,remainder__R,remainder__U,remainder__W,remainder___legendary,remainder___booster,remainder___resrved,remainder__text_embedding,remainder__types_embedding,remainder__subtypes_embedding,remainder__keywords_embedding
0,-0.787786,-0.818064,0.83164,-0.74473,1.0,0.0,0.0,0.0,0.0,0,...,0,0,1,0,1,0,"[-0.51127124, 0.7780097, -0.035495322, 1.00399...","[0.14906012, -0.21770309, 0.05100936, 0.053071...","[-0.37045395, 0.18439247, 0.06258764, -0.54133...","[-0.0070120296, -0.12131895, 0.020730816, -0.2..."
1,-0.237532,1.302998,0.930864,0.397826,1.0,0.0,0.0,0.0,0.0,0,...,0,1,0,0,1,0,"[-1.2777522, 0.23359896, 0.013131049, 1.333495...","[-0.20235693, -0.052378323, -0.3317952, -0.071...","[-0.27447513, -0.0665222, 0.20202424, -0.36353...","[0.09644347, -0.18145148, -0.3692048, 0.109952..."
2,1.963486,1.833263,0.967534,0.969104,0.0,1.0,0.0,0.0,0.0,1,...,1,1,0,1,0,0,"[-0.9803241, 0.21962857, 0.011534677, 0.834389...","[-0.20235693, -0.052378323, -0.3317952, -0.071...","[-0.10581706, -0.22339688, 0.15656523, -0.4000...","[0.12041464, -0.14207205, -0.083954014, 0.0695..."


In [100]:
processed_df =processed_df.astype({
    'num__power': float,
    'num__toughness': float,
    'num__released_at': float,
    'num__cmc': float
})

In [101]:
processed_df = processed_df.astype({
    "cat__rarity_common": int,
    "cat__rarity_mythic": int,
    "cat__rarity_rare": int,
    "cat__rarity_special": int,
    "cat__rarity_uncommon": int,
    "remainder__B": int,
    "remainder__C": int,
    "remainder__G": int,
    "remainder__N": int,
    "remainder__R": int,
    "remainder__U": int,
    "remainder__W": int,
    "remainder___legendary": int,
    "remainder___booster": int,
    "remainder___resrved": int
})

In [102]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25489 entries, 0 to 25488
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   num__power                     25489 non-null  float64
 1   num__toughness                 25489 non-null  float64
 2   num__released_at               25489 non-null  float64
 3   num__cmc                       25489 non-null  float64
 4   cat__rarity_common             25489 non-null  int64  
 5   cat__rarity_mythic             25489 non-null  int64  
 6   cat__rarity_rare               25489 non-null  int64  
 7   cat__rarity_special            25489 non-null  int64  
 8   cat__rarity_uncommon           25489 non-null  int64  
 9   remainder__B                   25489 non-null  int64  
 10  remainder__C                   25489 non-null  int64  
 11  remainder__G                   25489 non-null  int64  
 12  remainder__N                   25489 non-null 

In [103]:
indexes = processed_df[processed_df['remainder__text_embedding'].isna()].index

In [104]:
processed_df.drop(indexes, inplace=True)
processed_df.reset_index(drop=True, inplace=True)

In [117]:
y.drop(indexes, inplace=True)
y.reset_index(drop=True, inplace=True)

In [118]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25380 entries, 0 to 25379
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   num__power                     25380 non-null  float64
 1   num__toughness                 25380 non-null  float64
 2   num__released_at               25380 non-null  float64
 3   num__cmc                       25380 non-null  float64
 4   cat__rarity_common             25380 non-null  int64  
 5   cat__rarity_mythic             25380 non-null  int64  
 6   cat__rarity_rare               25380 non-null  int64  
 7   cat__rarity_special            25380 non-null  int64  
 8   cat__rarity_uncommon           25380 non-null  int64  
 9   remainder__B                   25380 non-null  int64  
 10  remainder__C                   25380 non-null  int64  
 11  remainder__G                   25380 non-null  int64  
 12  remainder__N                   25380 non-null 

In [119]:
X_numerical = processed_df[['num__power', 'num__toughness', 'num__released_at', 'num__cmc']]
X_categorical = processed_df[['remainder___legendary', 'remainder___booster', 'remainder___resrved',
                              'cat__rarity_common', 'cat__rarity_mythic', 'cat__rarity_rare', 'cat__rarity_special', 'cat__rarity_uncommon',
                              'remainder__B', 'remainder__C', 'remainder__G', 'remainder__N', 'remainder__R', 'remainder__U', 'remainder__W']]
X_text = processed_df[['remainder__text_embedding']]
X_subtypes= processed_df[['remainder__subtypes_embedding']]
X_types = processed_df[['remainder__types_embedding']]
X_keys = processed_df[['remainder__keywords_embedding']]

In [120]:
X_numerical.shape, X_categorical.shape, X_text.shape, X_subtypes.shape, X_types.shape, X_keys.shape

((25380, 4), (25380, 15), (25380, 1), (25380, 1), (25380, 1), (25380, 1))

In [136]:
type(X_text['remainder__text_embedding'][0])

numpy.ndarray

In [144]:
X_text['remainder__text_embedding'] = X_text['remainder__text_embedding'].apply(lambda x: np.array(x).flatten() if isinstance(x, list) or isinstance(x, np.ndarray) else x)

/var/folders/tm/jmhp8xvj4_d4hh_01cf1jt_r0000gn/T/ipykernel_12472/1709872702.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_text['remainder__text_embedding'] = X_text['remainder__text_embedding'].apply(lambda x: np.array(x).flatten() if isinstance(x, list) or isinstance(x, np.ndarray) else x)


In [123]:
X_numerical_train, X_numerical_test, X_categorical_train, X_categorical_test, X_text_train, X_text_test,X_subtypes_train, X_subtypes_test, X_types_train, X_types_test, X_keys_train, X_keys_test, y_train, y_test = train_test_split(
    X_numerical, X_categorical, X_text, X_subtypes, X_types, X_keys, y, test_size=0.30, random_state=42
)

In [124]:
def initialize_model():

    model_numerical = Sequential([
        Input(shape=(4,), name='input_numerical'),
        Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(8, activation='relu')
    ], name='model_numerical')

    model_categorical = Sequential([
        Input(shape=(15,), name='input_categorical'),
        Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dropout(0.2),
        Dense(16, activation='relu')
    ], name='model_categorical')

    model_text = Sequential([
        Input(shape=(300,), name='input_text'),
        Dropout(0.2),
        Dense(256, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dense(128, activation='relu'),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(256, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
    ], name='model_text')


    model_type = Sequential([
        Input(shape=(100,), name='input_type'),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
    ], name='model_type')


    model_subtype = Sequential([
        Input(shape=(100,), name='input_subtype'),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(32, activation='relu')
    ], name='model_subtype')


    model_keys = Sequential([
        Input(shape=(100,), name='input_keys'),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(32, activation='relu')
    ], name='model_keys')


    concatenated = Concatenate()([
        model_numerical.output,
        model_categorical.output,
        model_text.output,
        model_type.output,
        model_keys.output,
        model_subtype.output
    ])

    output = Dense(1, activation='linear', name='output')(concatenated)


    model = models.Model(inputs=[model_numerical.input, model_categorical.input, model_text.input, model_type.input, model_keys.input, model_subtype.input], outputs=output)

    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['mse', 'accuracy'])

    return model


In [125]:
model = initialize_model()

In [126]:
def get_history():
    es = EarlyStopping(patience=10, restore_best_weights=False)

    train_data = {
        'input_numerical': X_numerical_train,
        'input_categorical': X_categorical_train,
        'input_text': X_text_train,
        'input_type': X_types_train,
        'input_subtype': X_subtypes_train,
        'input_keys': X_keys_train
    }

    test_data = {
        'input_numerical': X_numerical_test,
        'input_categorical': X_categorical_test,
        'input_text': X_text_test,
        'input_type': X_types_test,
        'input_subtype': X_subtypes_test,
        'input_keys': X_keys_test
    }

    history = model.fit(
        train_data,
        y_train,
        epochs=20,
        batch_size=16,
        validation_data=(test_data, y_test),
        callbacks=[es],
        verbose=1
    )

    return history


In [127]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_text (InputLayer)     [(None, 300)]                0         []                            
                                                                                                  
 dropout_13 (Dropout)        (None, 300)                  0         ['input_text[0][0]']          
                                                                                                  
 input_numerical (InputLaye  [(None, 4)]                  0         []                            
 r)                                                                                               
                                                                                                  
 dense_32 (Dense)            (None, 256)                  77056     ['dropout_13[0][0]']    

In [128]:
get_history()

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:
test_data = {
        'input_numerical': X_numerical_test,
        'input_categorical': X_categorical_test,
        'input_text': X_text_test,
        'input_type': X_types_test,
        'input_subtype': X_subtypes_test,
        'input_keys': X_keys_test
    }
model.evaluate(test_data, y_test)

239/239 [==============================] - 0s 2ms/step - loss: 1885.5562 - mse: 1881.6256 - accuracy: 2.6154e-04


[1885.55615234375, 1881.6256103515625, 0.00026154046645388007]

In [ ]:
no_prices_df = df[df['usd'].isna()]